In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('data/Tampa_mexican_restaurants.csv')
df = df.drop(columns=['state', 'latitude', 'longitude','address'])

#remove html
from bs4 import BeautifulSoup

def remove_html_tags(html):
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text()

df.loc[:, 'cleaned_text'] = df['text'].apply(remove_html_tags)

# remove punctuation
import re

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

df.loc[:, 'cleaned_text'] = df['text'].apply(remove_punctuation)

# remove phone numbers
def remove_phone_numbers(text):
    pattern = r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b'
    return re.sub(pattern, '', text)

df.loc[:, 'cleaned_text'] = df['text'].apply(remove_punctuation)

/tmp/ipykernel_813/762684079.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, "html.parser")


In [13]:
business_review_counts = df['business_id'].value_counts()

# only leave restaurants with more than 100 reviews
business_ids_with_100plus_reviews = business_review_counts[business_review_counts > 100].index

filtered_df = df[df['business_id'].isin(business_ids_with_100plus_reviews)]

In [14]:
filtered_df.loc[:, 'text'] = filtered_df['text'].str.lower()

In [15]:
import spacy
from collections import Counter
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

nlp = spacy.load("en_core_web_sm")

def tokenize_and_lemmatize(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_punct]
    return tokens

def remove_stopwords(tokens):
    return [token for token in tokens if token not in stop_words]

def create_ngrams(token_list, nb_elements):
    ngrams = zip(*[token_list[index_token:] for index_token in range(nb_elements)])
    return [" ".join(ngram).strip() for ngram in ngrams]

def frequent_words(list_words, ngrams_number=1, number_top_words=10):
    frequent = []
    if ngrams_number == 1:
        pass
    elif ngrams_number >= 2:
        list_words = create_ngrams(list_words, ngrams_number)
    else:
        raise ValueError("number of n-grams should be >= 1")
    counter = Counter(list_words)
    frequent = counter.most_common(number_top_words)
    return frequent

# Apply tokenization, lemmatization, and stopword removal to the 'text' column
filtered_df['token'] = filtered_df['cleaned_text'].apply(tokenize_and_lemmatize)
filtered_df['filtered_token'] = filtered_df['token'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipykernel_813/1191511740.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['token'] = filtered_df['cleaned_text'].apply(tokenize_and_lemmatize)
/tmp/ipykernel_813/1191511740.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['filtered_token'] = filtered_df['token'].apply(remove_stopwords)


In [16]:
# Get the most frequent bigrams (2-grams)
most_frequent_bigrams = frequent_words(filtered_df['filtered_token'].explode().dropna(), ngrams_number=2, number_top_words=10)

for bigram, count in most_frequent_bigrams:
    if bigram:
        print(f"{bigram}:{count}")

mexican food:3232
food:3035
good:2312
ca nt:2037
come back:1917
go back:1906
order:1826
service:1624
chip salsa:1580
taco:1550


In [18]:
most_frequent_bigrams = frequent_words(filtered_df['filtered_token'].explode().dropna(), ngrams_number=3, number_top_words=10)

for bigram, count in most_frequent_bigrams:
    if bigram:
        print(f"{bigram}:{count}")

good mexican food:571
ca nt wait:558
authentic mexican food:555
definitely come back:355
mexican food:309
ca nt:275
food good:260
chip salsa:243
definitely go back:239
nt wait go:229
